# Newton-Raphson Method

## $$ x_n\,=\,x_{n-1} \, - \, \frac{f(x_{n-1})}{ f' (x_{n-1}) }\,\,\,\,\, \forall \,\,n\,\geq 1 $$ 

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pandas_datareader as pdr

from scipy.stats import norm
from dateutil import relativedelta
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
def imp_vol_cal (cp_flag, option_price, S0, K, T, r, q=0):
    
    log = np.log
    sqrt = np.sqrt
    exp = np.exp
    n = norm.pdf
    N = norm.cdf

    def find_price(cp_flag, S0, K, T, r, vol, q=0):
        
        d1 = (log(S0/K) + (r - q + (vol**2)/2) * T) / (vol * sqrt(T))
        d2 = d1 - vol*sqrt(T)

        if cp_flag == 'c':
            price = S0 * N(d1) - K*exp(-r*T)*N(d2)
        
        else:
            price = K * exp(-r*T)*N(-d2) - S0*exp(-q*T)*N(-d1)

        return price

    def vega_func (cp_flag, S0, K, T, r, vol, q=0):

        d1 = (log(S0/K) + (r + (vol**2)/2) * T) / (vol * sqrt(T))

        return S0 * sqrt(T) * n(d1)

    '''use above functions with Newton_Raphson_method'''

    MAX_ITERATIONS = 100
    ERROR = 10e-6

    VOL = 0.5 #Initial Volatility

    for NUM_ITERATIONS in range(0, MAX_ITERATIONS):
        
        price = find_price(cp_flag, S0, K, T, r, VOL, q=0)
        vega = vega_func(cp_flag, S0, K, T, r, VOL, q=0)

        diff = option_price - price

        if abs(diff) > ERROR:
            VOL = VOL + diff/vega  #price? diff?

        else:
            return print(f'IV = {round(VOL, 4) * 100}%', f'Iteration Num = {NUM_ITERATIONS}', f'Error = {diff}\n', sep='\n')

    return print(f'** Calculation Failed (result below shows the best answer) **', f'IV = {VOL}', f'Iteration Num = {NUM_ITERATIONS}', f'Error = {diff}\n', sep='\n')

        

$$ S_0 \times \sqrt{T} \times \frac{e^{-x^2/2}}{\sqrt{2\pi}}$$

# Assignment

## 1) Implied Volatility Calculate

In [5]:
S0 = 273.55
K = 257.5
T = 10/360
r = 0.0155

imp_vol_cal('c', 16.15, S0, K, T, r)

S0 = 273.55
K = 260
T = 10/360
r = 0.0155

imp_vol_cal('c', 14, S0, K, T, r)

IV = -inf%
Iteration Num = 15
Error = nan

IV = 21.66%
Iteration Num = 4
Error = -2.7219121534471924e-06



Note that

$$ \text{Black-Scholes Solution} : C(S_0, T) \,=\, S_0 e^{-qT}N(d_1)\,-\,Ke^{-rT}N(d_2)$$

$$ d_1\,=\,\frac{1}{\sigma\sqrt{T}} \left[ \, \ln{\frac{S_0}{K}} \, + \, \left( r \,-q\,+\frac{\sigma^2}{2}\right) T\right]$$

$$ d_2\,=\,\frac{1}{\sigma\sqrt{T}} \left[ \, \ln{\frac{S_0}{K}} \, + \, \left( r \,-q\,-\frac{\sigma^2}{2}\right) T\right]$$

$\sigma \rightarrow \infin$ 할때, $d_1, d_2 \rightarrow \infin, -\infin$ 하게 된다. <br>
이때, 정규분포의 누적분포함수($N(\cdot)$)가 단조증가함수 이므로, $N(d_1), N(d_2) = 1, 0$ 이 된다 <br>




## 2) Thomas Algorithm

### Answer

In [51]:
def thomas(a,b,c,d):
    """ A is the tridiagnonal coefficient matrix and d is the RHS matrix"""
    N = len(a)
    cp = np.zeros(N,dtype='float64') # store tranformed c or c'
    dp = np.zeros(N,dtype='float64') # store transformed d or d'
    X = np.zeros(N,dtype='float64') # store unknown coefficients
    
    # Perform Forward Sweep
    # Equation 1 indexed as 0 in python
    cp[0] = c[0]/b[0]  
    dp[0] = d[0]/b[0]
    # Equation 2, ..., N (indexed 1 - N-1 in Python)
    for i in np.arange(1,(N),1):
        dnum = b[i] - a[i]*cp[i-1]
        cp[i] = c[i]/dnum
        dp[i] = (d[i]-a[i]*dp[i-1])/dnum
    
    # Perform Back Substitution
    X[(N-1)] = dp[N-1]  # Obtain last xn 

    for i in np.arange((N-2),-1,-1):  # use x[i+1] to obtain x[i]
        X[i] = (dp[i]) - (cp[i])*(X[i+1])
    
    return print('The Values of x are:', np.round(X, 3), sep=' ')

In [52]:
a = [0, -1, 0.5, -2, 1, 1]
b = [2, 3, 2, 4, 2, 3]
c = [1, 1, 0.5, 0.5, 0.3, 0]
d = [2, -1, 0.5, 2.3, -5, 0.25]

thomas(a, b, c, d)

The Values of x are: [ 1.001e+00 -2.000e-03  7.000e-03  9.730e-01 -3.157e+00  1.136e+00]


In [54]:
a = [0, -2, 0.5, -2, 1, 1]
b = [1, 1, 1, 1, 0.2, 1]
c = [2, 1, 0.5, 0.5, 0.3, 0]
d = [2, -1, 0.5, 2.3, -5, 0.25]

thomas(a, b, c, d)

The Values of x are: [  1.596   0.202   1.99   -3.182  18.926 -18.676]


정답

$$\begin{pmatrix}1.596& 0.202&1.990&-3.182&18.926&-18.676 \end{pmatrix}$$

### Trial

In [53]:
def thomas_algorithm(a, b, c, d):
    'a = lower diagonal one vector, b = middle diagonal one vector, c = upper diagonal one vector, d = solved vector'

    N = len(d):

    if N == 1:

        c_b = c/b
        d_b = d/b

    else:

SyntaxError: invalid syntax (2018509806.py, line 4)

In [27]:
A = np.array([[1,2,0,0,0,0], [-2,1,1,0,0,0], [0, 0.5, 1, 0.5, 0, 0], [0, 0, -2, 1, 0.5, 0], [0, 0, 0, 1, 0.2, 0.3], [0, 0, 0, 0, 1, 1]])
B = np.array([2, -1, 0.5, 2.3, -5, 0.25])

(6, 6)

In [28]:
np.linalg.inv(A) @ B

array([  1.59609375,   0.20195312,   1.99023438,  -3.18242188,
        18.92578125, -18.67578125])

In [ ]:
log = np.log
sqrt = np.sqrt
exp = np.exp

n = norm.pdf
N = norm.cdf


def bs_price(cp_flag,S,K,T,r,v,q=0.0):

    d1 = (log(S/K)+(r+v*v/2.)*T)/(v*sqrt(T))
    d2 = d1-v*sqrt(T)

    if cp_flag == 'c':
        price = S*exp(-q*T)*N(d1)-K*exp(-r*T)*N(d2)
    
    else:
        price = K*exp(-r*T)*N(-d2)-S*exp(-q*T)*N(-d1)
    
    return price

def bs_vega(cp_flag,S,K,T,r,v,q=0.0):

    d1 = (log(S/K)+(r+v*v/2.)*T)/(v*sqrt(T))
    
    return S * sqrt(T)*n(d1)

def find_vol(target_value, call_put, S, K, T, r):
    MAX_ITERATIONS = 100
    PRECISION = 10e-6

    sigma = 0.5
    idx = 0
    for i in range(0, MAX_ITERATIONS):
        idx += 1
        price = bs_price(call_put, S, K, T, r, sigma)
        vega = bs_vega(call_put, S, K, T, r, sigma)

        price = price
        diff = target_value - price  # Our roots >> Checking Rate of Convergence (diff means error)

        if (abs(diff) < PRECISION):
            return print(f'{sigma} and Iteration {idx}')
        sigma = sigma + diff/vega # f(x) / f'(x)

    # Value not found , Return to your best guess so far 
    return sigma

def option_date_creator (week):
    '''insert int, like "1" = 1week later option from now'''

    today_date = dt.datetime.today()
    option_date = today_date + relativedelta.relativedelta(weekday=4) + dt.timedelta(weeks=week)
    option_date = option_date.strftime('%Y-%m-%d')

    return option_date

In [ ]:
find_vol(7.85, 'c', 415.89, 420, 42/365, 0.0092)

0.16933911677691518 and Iteration 3


In [ ]:
find_vol(8.13, 'p', 415.89, 420, 42/365, 0.0092)

0.10832869019681027 and Iteration 4


In [ ]:
Ticker = 'AAPL'
call_chain = yf.Ticker('AAPL').option_chain(option_date_creator(1)).calls
ticker_S = yf.download('AAPL', progress=False).Close[-1]
rf = pdr.DataReader('DGS10', 'fred').values[-1][0] / 100

In [ ]:
call_chain

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220909C00070000,2022-08-30 19:54:40+00:00,70.0,88.35,0.0,0.0,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
1,AAPL220909C00075000,2022-08-26 19:40:58+00:00,75.0,89.05,0.0,0.0,0.0,0.0,24.0,0,0.000010,True,REGULAR,USD
2,AAPL220909C00080000,2022-08-30 13:48:16+00:00,80.0,81.30,0.0,0.0,0.0,0.0,4.0,0,0.000010,True,REGULAR,USD
3,AAPL220909C00085000,2022-08-30 13:58:22+00:00,85.0,76.25,0.0,0.0,0.0,0.0,4.0,0,0.000010,True,REGULAR,USD
4,AAPL220909C00090000,2022-08-30 14:02:47+00:00,90.0,71.35,0.0,0.0,0.0,0.0,5.0,0,0.000010,True,REGULAR,USD
5,AAPL220909C00095000,2022-08-30 13:52:39+00:00,95.0,66.65,0.0,0.0,0.0,0.0,4.0,0,0.000010,True,REGULAR,USD
6,AAPL220909C00100000,2022-08-30 13:46:53+00:00,100.0,61.30,0.0,0.0,0.0,0.0,5.0,0,0.000010,True,REGULAR,USD
7,AAPL220909C00105000,2022-08-30 16:50:37+00:00,105.0,53.85,0.0,0.0,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
8,AAPL220909C00110000,2022-08-30 14:00:37+00:00,110.0,51.25,0.0,0.0,0.0,0.0,9.0,0,0.000010,True,REGULAR,USD
9,AAPL220909C00115000,2022-08-26 19:40:58+00:00,115.0,49.10,0.0,0.0,0.0,0.0,3.0,0,0.000010,True,REGULAR,USD


In [ ]:
ticker_S

158.91000366210938

In [ ]:
call_price[idx]

12.85

In [ ]:
option_maturity = option_date_creator(1)
ticker_T = int((dt.datetime.strptime(option_maturity, "%Y-%m-%d") - dt.datetime.today()).days) / 365

call_K = call_chain['strike'].to_list()
call_price = call_chain['lastPrice'].to_list()

idx = 20
find_vol(call_price[idx], 'c', ticker_S, call_K[idx], ticker_T, rf) * 100

nan